In [1]:
from mistralai import Mistral
import os
MISTRAL_API_KEY="6AqHoOaGkd1dYqIFA9iwWbRAfH6oBoLn"

In [2]:
class MistralAPI:
    """
    A client for interacting with the MistralAI API.

    Attributes:
        client (Mistral): The Mistral client instance.
        model (str): The model to use for queries.
    """

    def __init__(self, model: str) -> None:
        """
        Initializes the MistralAPI with the given model.

        Args:
            model (str): The model to use for queries.

        Raises:
            ValueError: If the MISTRAL_API_KEY environment variable is not set.
        """
        api_key = os.getenv("MISTRAL_API_KEY")
        if not api_key:
            raise ValueError(
                "No MISTRAL_API_KEY as environment variable, please set it!"
            )
        self.client = Mistral(api_key=api_key)
        self.model = model

    def query(self, query: str, temperature: float = 0.5) -> str:
        """
        Sends a query to the MistralAI API and returns the response.

        Args:
            query (str): The input query to send to the model.
            temperature (float, optional): The temperature parameter for controlling
                                          the randomness of the output. Defaults to 0.5.

        Returns:
            str: The response from the API.
        """
        chat_response = self.client.chat.complete(
            model=self.model,
            temperature=temperature,
            messages=[
                {
                    "role": "user",
                    "content": query,
                },
            ],
        )
        return chat_response.choices[0].message.content

In [3]:
def build_prompt(context:str, query: str) -> str:
    """
    Builds a prompt string for a conversational agent based on the given context and query.

    Args:
        context (str): The context information, typically extracted from books or other sources.
        query (str): The user's query or question.

    Returns:
        str: The formatted prompt string.
    """
    context_joined = '\n'.join(context)
    return f"""Tu es l'agent conversationnel de Ricco. Tu tires ton savoir de ses livres.
    Ton rôle est d'aider les élèves dans leurs tâches  de statistique et Machine Learning. 
    Tu disposes de la section "Contexte", des extraits du livres, pour t'aider à répondre aux questions.
    Sois le plus instructif possible.

    # Contexte: 
    {context_joined}

    # Question:
    {query}

    # Réponse:
    """

In [4]:
# Set the MISTRAL_API_KEY environment variable
os.environ["MISTRAL_API_KEY"] = "6AqHoOaGkd1dYqIFA9iwWbRAfH6oBoLn"
# print(ministral8b.query("Réalise une analyse de sentiment sur la phrase suivante : 'c est le meilleur restaurant du quartier, mais les tarifs sont explosifs', et renvoie-moi un dictionnaire python au format {'label': l, 'subjectivity': s} avec : l une indication de polarité parmi les choix suivants : positif, négatif, neutre. S est une note de subjectivité variant de 0 (complètement subjectif) à  1 (complètement objectif). Je ne veux pas de code, mais un résultat au format indiqué."))

In [5]:
import os

review1 = """Agréable petit restaurant à la déco typique,la cuisine est bonne et faites maison. Tenu par un adorable couple,le service est rapide et agréable.Je le recommande vivement.A essayer la soupe aux nouilles froide. Ce restaurant est une magnifique découverte. La cuisine y est excellente et le service top. Nous avons beaucoup apprécié et y retournerons sans hésiter. En plus, ils avaient des cookies au matcha... un délice. Lendroit ne paie pas de mine mais les plats sont faits maison, les gyoza sont excellents, le ramen aussi et jai testé le tiramisu au thé macha, très bon  En plus les prix sont raisonnables, je vous le conseille Un très bon moment, les plats sont très bons et le service très agréable. Le prix est tout à fait raisonnable compte tenu de la qualité des plats 1ère et dernière fois  Jai commandé un Donbury shakedon un grand bol de riz avec du saumon. Alors comment dire  Une honte, le saumon censé être sauté à la poêle et caramélisé est en réalité constitué de tous les bas morceaux écartés lors de la confection des sushis dans ce même restaurant. Vous savez, les morceaux gris qui sont retirés du saumon parce quils ont mauvais goût  Eh bien, tous ces morceaux dont personne ne veux vous sont servis sur un grand bol de riz trop cuit et sans saveur. Le saumon nétait en aucun cas caramélisé  Ajoutés à cela des petits pois surgelés et quelques rares carottes râpées en sachet. Limpression dêtre dans une mauvaise cantine scolaire. Quel mépris pour le client  Ce nétait vraiment pas bon Situé dans le quartier des Brotteaux, lizakaya Kuma est simple daccès et facilement repérable malgré sa devanture discrète. Lintérieur est tout bonnement sublime avec une déco épurée typiquement japonaise. La cuisine na vraiment rien à envier aux autres restaurants japonais. Les plats sont réalisés sur place, juste derrière le comptoir, et servis tout chaud  Nous avons dailleurs lembarras du choix et tout le monde y trouve son compte. Tellement bon quon a hésité pour commander un autre hors dœuvre vive les takoyakis  Quant aux desserts, très bon également, avec un bon accent sur les desserts au thé vert. Petite note pour les amateurs de sake alcool de riz japonais  le Kuma propose une palette de choix et en fait sa spécialité. Venez déguster des spécialités japonaises pour le plus grand plaisir de vos papilles  Vous ne serez pas déçus  Attention aux gourmands etou gros mangeurs, la note peut monter et se saler très rapidement... Une belle soirée entre amis. Nous nous sommes régalés. Des plats typiques très goûteux, des bières très agréables et en grand format. Je recommande le tiramisu au thé vert, un délice. Le service était parfait, un tantinet trop rapide toutefois. Des serveurs très agréables. Un petit cafouillage à la caisse au moment de payer, rien de bien méchant. Nous recommandons. Je conseille fortement le sushi de saumon  Les serveurs sont très sympas aussi. Jai déjà commandé plusieurs fois en livraison, et cest toujours bien emballé. Après, cest toujours mieux dy aller manger sur place car on trouve des bonnes propositions sur lardoise. Quel régal Les entrées sont délicieuses dans une cuisine typiquement japonaise dans prétention Les plats étaient copieux et exquis Un ami ayant vécu au Japon na retrouvé le vrai goût de la cuisine japonaise que dans ce restaurant donc je recommande Installée avec gentillesse au comptoir, la serveuse a su merveilleusement me conseiller pour la bière mais aussi pour le thé. Concernant les plats, les gyozas étaient parfaitement cuits, fondants et juteux comme il faut. Le Teriyaki-don était lui aussi succulent et très bien accompagné avec sa soupe miso. Le Doriyaki avec sa boule de glace fermait parfaitement la marche accompagné dun très bon thé qui fut une belle découverte. Bref  Tout était parfait. Je reviendrais avec plaisir goûter le reste de la carte. Les bonnes critiques sur trip advisor nous ont fait pousser les portes de ce petit restaurant japonais. Super accueil gracieux et service efficace des deux chargées de la prise de commande et du service. De bon conseil, elles nous ont fait testé le saké. Côté cuisine, cest bon mais sans éclats gustatifs. Les plats sont copieux et doguibags possible pour les restes. Le restaurant est vite pleine réservation fortement recommandée. Nos enfants voulaient goûter des ramen effet série Naruto  donc nous avons opté pr ce restaurant lyonnais. Quel bon moment On sest régalé et les serveuses sont très professionnelles et très sympathiques  on a pu bien échanger sur la nourriture japonnaise et même sur le manga. Continuez à nous régaler. Repas très bon, personnel accueillant et souriant  endroit cosy et bien decoré. on recommande, on a passé un très bon moment. En restaurant avec une belle décoration, et des assiettes très bien présentées. Nous avons pris un ramen curry et un ramen chashu. Le tonkatsu était très bien réalisé et jai tout simplement adoré le tofu soyeux en entrée, une entrée froide parfaite en cette chaleur. Nous navons pas pris de saké mais ils ont une très belle sélection. Quelle belle surprise  Un accueil chaleureux, les cuisines des lentrée montrent les chefs au boulot. Carte qui nest pas surchargée donc cest bon signe  Un choix de bières japonaises fort sympathiques . Nous avons pris 2 ramens  excellents  bouillon aux goûts équilibrés, nouilles extras   puis un the matcha gourmand  vraiment très très bien . On sest régalé  Super rapport qualité prix  Bravo bravo Très bon japonais lyonnais, nous avons été conquis autant par les plats que par le service  Lendroit est très sympa mais pensé à réserver. Nous avons apprécié tous les plats que nous avons pris, et cela nous a fait découvrir la gastronomie japonaise, nous qui ny connaissions pas grand chose  Les gyozas aux légumes étaient excellents, fondants et savoureux, le ramen et le légumes udon également  Il faut goûter les desserts, ce nest pas des goûts et des textures dont on a lhabitude  Nous avons passé un bon moment, et les prix sont très corrects pour la qualité et la quantité Restaurant au jolie décors japonais. Les serveuses et les cuisiniers chinois et non japonais sont très sympathiques. Malheureusement la nourriture nest pas à la hauteur. La soupe est insipide. Lœuf qui devrait être mollé est un œuf dur. Le thé vert est servi avec de leau bouillante donc amer alors quil devrait être infusé peu de temps dans de leau à 70-80 degrés. Décevant mais pas trop cher 13 euros environ le plat Accueil très sympathique et souriant. Les personnes prennent le temps dexpliquer les plats et avec sourire. Nous avons pris 3 petits plats a partager et 2 plats de nouilles. Attention c très copieux. Tout était très bon et parfaitement présenté. Je suis pas un grand connaisseur donc je peux juste dire que cela nous a beaucoup plu. Nous reviendrons. Dès que nous passons la porte du restaurant nous nous croyons de suite au Japon  Cuisine excellente  et personnel au top  Nous recommandons absolument  Nous reviendrons Très bon accueil, la nourriture est excellente et un joli lieu pour une belle découverte du saké À recommander Nous y reviendrons On a tout simplement adoré. Cétait excellent du début jusquà la fin. Service irréprochable. Lieu cosy. Une très très bonne adresse que nous recommandons vivement  Resa conseillée. Cétait vraiment délicieux. Je conseille les sobas, lomelette et les boulettes de poulpes. Déco incroyable, tout était parfait Les plats servis sont copieux et de bonne qualité. Jai adoré mon plat. La carte est variée et il y en a pour tous les goûts. Niveau tarifs, rien à redire. Petite mention coup de coeur pour le dessert à base de matcha. Très bon et pas trop sucré. Le personnel est très sympathique et lambiance chouette  Hésitez pas à réserver car la salle est petite. Hâte dy retourner lors de mon prochain passage sur Lyon. Très bonne cuisine et personnel très sympathique  Ladresse vaut ses compliments. Par contre la réservation est obligatoire car le restaurant est petit Une belle surprise que cet izakaya proposant des tapas japonais ou plats aux choix. Jai kpte pour un assortiment de plusieurs petites salades et spécialités , mention spéciale pour la salade d encornets et boulettes de poulpe. Mon mari a testé le shashu ramen, une soupe de nouilles au porc délicieuse. Très belle carte de sakés également. Excellent japonais comme au Japon. Cest beau et bon. Cétait la quatrième fois que nous venions et nous reviendrons  Très bon rapport qualité-prix Les plats sont juste délicieux, juste à fermer les yeux et on se retrouve au Japon tellement les saveurs sont là  Petite salle à létage, restaurant pas très grand, juste ce quil faut. Nous reviendrons avec plaisir, Merveilleux moment culinaire  Amateurs de soupe sy rendre sennuyer plus tarder Petit endroit mais chaleureux et charmant Un véritable régale. Nous y sommes allé à 2 et tout était parfait. De lentrée jusquau dessert en passant par le service  Mention spéciale pour le Tiramisu the matcha  Merci à vous Une tuerie. Jai retrouvé les saveurs que javais découvert sur Tokyo.  Jy retournerai les yeux fermés  En revanche il est vrai quil faut prévoir un certain budget. Une cinquantaine deuros pour 2 personnes sans desserts, mais le goût et les proportions les valent Excellent. Tous les plats sont incroyablement bons. Le serveur était très agréable et a beaucoup fait rire notre table. De lentrée au dessert, tout était délicieux. Je recommande absolument Très belle découverte  référence sur Lyon  Cela fait déjà plusieurs que je my rends, et je ne prends même pas la peine de donner mon avis. Quel indignité.. Je nai jamais été déçu et cest toujours un réel plaisir dy retourner. Une vrai escapade culinaire Japonaise. Petit conseil, les Ramens se mangent bruyamment en faisant de gros SLURPS pour faire refroidir le bouillon servi très chaud avant quil narrive dans la bouche. Plus vous faîtes de bruit, plus cela signifie que vous vous régalez, alors ne vous gênez pas Nous pensions être dépaysés en allant dans ce restaurant. Mais en entrant, laccueil de 2 jeunes en short nous a un peu refroidis. De plus pour avoir le menu il faut scanner un qr code. Jamais vu ça. Et si nous navions pas de téléphone  Le menu est mal foutu on y comprend rien. Nous avons commandé gyozas, boulettes de poulpe et beignets de crevette. Tout était délicieux mais je nai pas trop compris la pointe de mayonnaise dans les boulettes au poulpe. Mdr le tube de mayo sur le comptoir au passage. En plat jai été déçue par le porc tonkatsu. Sec et presque pas de sauce. Lautre plat au poulet était meilleur. Jaurais voulu manger une soupe avec des ramen mais comme on ny comprenait rien à ce menu je me suis trompée. Le tiramisu au matcha était bon mais pas assez sucré. Le prix des sakés est exhorbitant. Nous avons passé notre tour. Pas de cocktail alcoolisé sur leur carte dommage. Le prix nest cependant pas excessif mais les plats trop secs. Pour toutes ces raisons je mets 3 étoiles et nous ne reviendrons pas. Jy vais depuis plusieurs années avec ma famille ou bien des amis. Je nai jamais été déçue. Le personnel est top et la nourriture aussi. Je recommande vivement  Très sympathique le soir pour partager à plusieurs des petits plats et tester une bonne partie de la carte. Gros coup de coeur pour le tiramisu au matcha. A chaque visite nous sommes accueillis et nous ne sommes jamais déçus. Cest un plaisir de pouvoir conter sur cette adresse. Petit restau dans les rues de lyon 6, bien simpa avec petite ambiance japonaise dès quon rentre. Tous les plats et desserts sont typiques du japon ainsi que les boissons, et les prix sont raisonnables On y mange très bien, jai adoré les karaage avec le riz Et le service est top Equipe souriante et gentille. Je recommande ce bon restau japonais Toujours difficile de trouver un bon resto japonais et là... très belle surprise On retrouve vraiment les saveurs authentiques du Japon Les plats sont frais et délicieux. Je recommande a 100 Au niveau acceuil jai connu beaucoup mieux, Le cuisinier te gueule dessus d entrer, pour non port du masque alors que la distanciation sociale est respecter, de surcroit les serveurs qui laisse poireauter 10 min avant de te placer alors qu il n y pas grand monde, les plats sont bons mais les portions sont ridicules, une collation est necessaire avant daller dans ce restaurant, le tout pour un prix très elevé et cela sans dessert. Réputation surfaite, on trouve beaucoup mieux en japonnais sur Lyon, je ne retourne pas dans ce restaurant Les plats sont savoureux. Les udons sont un délices. Bonne expérience, serveurs très gentilles, service ultra rapide. . Adresse très sympa, mets et bières de qualité. On sort des sushis maki ... habituels Beau voyage culinaire Très très bon restaurant japonnais. Superbe accueil et lieu très sympa. Pensez à réserver car victime de son succès. Le restaurant est dépaysant, dès lentrée vous changez de continent. Laccueil est très sympathique. Les plats excellents. Et tout ça pour un prix raisonnable. Bref je recommande. une vraie expérience japonaise, un régal. Lentrée de tofu frais fut une superbe découverte pour moi et le ramen parfais. le service top également. Les plats sont frais, excellent, et assez copieux. Le cadre et le décor sont travaillé. Bref, a essayer. Le personnel accueillant et sympathique, très bonne cuisine...On sest régalé...Nous avons voyage le temps dun repas...on vous le recommande...il est préférable de réserver pour être sur davoir une table. Les plats sont vraiment bons et les prix sont très abordables Je recommande Mention spéciale pour le poulet teriyaki Restaurant de cuisine japonais, très fréquenté en ce midi, il faut réserver de préférence. Ambiance agréable et bonne cuisine qui change des traditionnels sushis et autres. Ce nest pas de la haute gastronomie japonaise mais cest bon et à des prix raisonnables. Bien entendu, grand choix de bières japonaises et de saké. Je vais chez Kuma Izakaya depuis un moment, quasiment une fois par mois  Je retrouve dans ce restaurant une ambiance typique et de bons plats japonais. Leur tiramisu au thé matcha est vraiment top Dépaysement au rendez vous, très très bonne cuisine japonaise, personnel accueillant et sympathique le seul hic... Laddition est un peu salé mais ça en vaut la peine Très bonne cuisine et ils ne lésinent pas sur la quantité, on peut dire quon est rassasié et je pèse mes mots en sortant du restaurant. Mais bon rapport qualité prix. La cuisine est traditionnelle et nous fait vraiment voyager, la deco du restaurant aussi. Même les desserts viennent du Japon, notemment leur tiramisu au thé vert. Lacceuil est moyen. Si le cuisinier na plus envie de bosser après 21h il suffit de lénoncer clairement ou modifier ses heures douverture à la place de gueuler depuis la cuisine quil ne cuisine plus à emporter puis de tenter de se justifier en évoquant le fait quil na plus de contenants pour emporter. Peut être que leur classement est justifié par la qualité de leur cuisine que je nai pas pu juger mais en tout cas certainement par laccueil du client Ne connaissant pas bcp la cuisine japonaise j ai bien apprécié et découvert des saveurs. Bon service. Cétait pour moi une découverte de la cuisine japonaise. J ai beaucoup aimé  On est loin des clichés de bar à sushi. Tout était savoureux et cuisiné avec des produits frais. Le service est efficace et discret. Je recommande Un restaurant japonais à la hauteur de nos espérances Tout était parfait. Un délice du début à la fin Je recommande vivement. Service rapide et chaleureux. la salade de poulpe était très bien assaisonnée, les ramens étaient top, bref la cuisine est de très bonne qualité. Je mets 4 étoiles car le service nétait pas très chaleureux, pas vraiment enthousiaste, pas à la hauteur de la cuisine. Jai eu le plaisir de déguster un Hiyayakko tofu frais avec purée dail, des takoyaki boulettes de poulpe, une salade shiitake salade de champignons et des gyoza aux légumes. Le tout à emporter  A recommander Très bon restaurant testé 2 fois avec une amie. Les plats sont dune grande finesse. Laccueil est chaleureux et la serveuse, toujours souriante, explique patiemment les plats et donne des recommandations. Allez y les yeux fermés  cest une excellent adresse lyonnaise Ici pas de sushis et cest tant mieux. Les plats sont excellents, avec du goût. Le lieu est tout petit mais intimiste. A decouvrir Un très bon restaurant de cuisine traditionnelle japonaise, pour un prix tout fait raisonnable  On sest vraiment régalées avec des saveurs authentiques et subtiles dans un cadre sympathique. Pour les gourmands, les plats de ramen et yakisoba sont vraiment copieux  Le service a été rapide et agréable. Le  original  un très très grand choix dalcools japonais  saké, liqueurs, bières... 4 ou 5 pages consacrées dans le menu  Ex  1 pers.  Liqueur dapéritif  2 dégustations dentrée  yakisoba ou ramen  dessert  35 Juste sublime car petit japonais traditionnel sans prétention mais tellement délicieux... Mon conjoint et moi avons pris les donburis à base de porc et moi de bœuf et ma belle fille les udons à la crevettes  Chacun était conquis de son plat et de son choix. Pensez bien à réserver Ce restaurant propose une carte variée. Ici pas de makis ou de sushis, mais une délicieuse cuisine japonaise variée. Coté boissons vous ne serez pas déçus puisque la carte comporte bières japonaises mais surtout de nombreux sakes. Laccueil et le service sont parfaits, le rapport qualité prix très bon. Première visite et une bonne surprise, excellent au niveau de laccueil, du service, du goût et de la quantité. Plat bon, raffiné et bien présenté. On peut voir la cuisine bien entretenu, petit restaurant, mais très bien. On y retournera cest sûr  Merci pour votre accueil et ces plats délicieux. Qualité et service top, très bien mangé, soupe et nouilles très bien cuisiné, acceuil, propreté, nous reviendrons De passage à Lyon nous avons fait une virée en centre ville et avons trouver ce petit restaurant très sympathique. La cuisine est un régale et le personnel adorable et quand au prix très correcte. Nous y reviendrons avec plaisir. Restaurant très accueillant, personnel sympathique. Le cadre nous invite au voyage. La carte change de lordinaire  pas de sushi, de maki ou autre sashimi  Dautres plats japonais que jai vraiment apprécié  Une très belle surprise Je reviens souvent dans ce restaurant japonais qui ne propose pas de sushi mais de véritables mets japonais très bons. Petit à petit jai essayé presque tous les plats et à chaque fois je nai jamais été déçue. Comme jai passé pas mal de temps au Japon, je trouve que les plats proposés sont proches de ceux qui se mangent au Japon, notamment les takoyaki. Les desserts ne sont pas mal, mais pas comme au Japon. Une soirée topissime  Sur réservation, nous avons mangé dans la salle à létage, très calme pourtant pleine et où le son résonne peu très très appréciable après une journée de boulot. Les tables peu nombreuses et bien espacées permettent dêtre vraiment tranquille. Le service était très bien, et les plats délicieux jai préféré les petites entrées à partager. Je navais plus faim pour les desserts, qui donnaient pourtant très envie, mais je compte bien y retourner. Bref, une adresse qui nous ramène au Japon en tout simplicité et que je recommande ...comme certains avis peuvent être divergents  Comme moi, vous vous en lisez un certain nombre pour vous faire une idée et décider si les négatifs ne font pas le poids avec les positifs. Alors, faisons simple  KUMA, cest un restaurant facile, simple, bon économique avec des saveurs qui changent des sushis. Après il y a les grands connaisseurs du Japon pour avoir vécu et qui affirment que ce nest pas de la cuisine japonaise et qui vont jusquà rechercher que le gérant est chinois...merci pour linfo, mais si vous voulez passer un moment sympa, tentez lexpérience. Un très bon restaurant japonais, cela change des restaurants de sushis. Les ramens étaient délicieuses. Voilà un restaurant japonais original, pas de sushi, maki ou autres poissons crus, mais des plats chauds venus directement de lempire du soleil levant. Toujours avec ma fille à la recherche du meilleur Japonais de Lyon, on a trouvé ici une perle rare. On a mangé au comptoir car cest plein le midi et il faut réserver normalement. deux petites places au comptoir pour nous, sans réservation super La qualité des plats est là, on mange vraiment bien et prix final 34 euros, à deux sans boisson, pas mal du tout. On conseille. Ambiance Izakaya inimitable avec une grande variété de petits plats tous excellents. Les plats principaux et déserts sont aussi dune grande qualité. Le tout pour une prix très abordable Je pense que la réputation de ce restaurant nest plus à faire vu la bonne note quil a. Honnêtement on sest vraiment régalé. Super rapport qualitéprix, bon accueil, repas plus que délicieux, pleins de saveurs et découverte des produits asiatiques. Allez y les yeux fermés Nous avons découvert ce restaurant il y a quelques semaines, à la recherche dun restaurant où nous pourrions manger des spécialités japonaises et navons pas été déçus  Nous avons pris gyoza et takoyakis en entrées, qui sont délicieux, tout comme les yakisoba et udon en plats. Les desserts sont aussi très bons tiramisu, dorayaki, le service est agréable et rapide et la décoration du restaurant est très belle. En bref, très bonne découverte, nous ne sommes pas de Lyon mais nous revenons avec plaisir. A recommander Conversation téléphonique agréable pour la réservation, sourires chaleureux à notre arrivée malgré un retard de 15mn, salle agréable. Que ce soit les entrées froides ou chaudes, le ramen ou les pâtes sautées aux crevettes, comme les desserts, cela sent et goutte les produits fait maison, de qualité. Nos amis à quatre pattes ne sont pas admis nous avions vérifié auparavant par tel. Nous y retournerons très certainement. Il faut oser pousser cette petite porte. Ça vaut le coup Les nouilles comme dans Naruto  Mon fils était trop heureux Nous nous sommes régalés. Déco typique. On adore Un petit restaurant style bistrot japonais qui sait allier plat traditionnelle simple de la cuisine japonaise Ramen, gyoza, salades, nouilles sautées, dombouri...et la dégustation de saké. Les prix sont très raisonnable et les plats délicieux. Chose rare, les desserts sont également à tomber. Le service est discret et attentionné. Très bon diner, typique et très bon rapport qualitéprix. Les entrées à partager cest très sympa, le service est très bon. Restaurant très propre. Un excellent restaurant qui propose des plats traditionnels comme au Japon. Bref si vous cherchez un restaurant japonais typique vous pouvez y aller les yeux fermés. On vient très souvent dans ce restaurant car entre les prix abordables et la qualité des produits, nous sommes conquis  Les takoyki , les gyoza et les ramen sont excellents mais alors le tiramisu au thé vert est carrément à tomber parterre . Je recommande Super restaurant  plats comme au Japon tous aussi bons les uns que les autres, service rapide et agréable. Nous y retournerons à coup sûr Jy suis allé à plusieurs reprises et à chaque fois cétait un délice Les plats sont excellents et copieux De laccueil à laddition tout sest très bien passé... Je recommande vivement Cadre cosy cest petit, bien penser à réserver  et accueillant. Un mélange de saveurs bien maîtrisé, tout en finesse. Nous avons adoré. Nous ne sommes pas trop sucré et avons goûté des parfums de glace subtiles, surprenants. Nous y retournerons. Très bon petit restaurant japonais, cuisine zéro sushi, une découverte culinaire dans un cadre simple et un peu petit mais avec une équipe efficace, et un joli choix de saké, jai pour ma part goûter le kambara, vraiment bien. Très bonne adresse que je recommande vivement. Très bon accueil, beau cadre et le repas est copieux. Très bon rapport qualité prix Laccueil est sympathique, le cadre bien. Grand choix de saké. Les plats sont savoureux pour un prix abordable. Bonne impression. Nous avons mangé en couple dans ce restaurant. Tout était parfait beau et bon. Bon accueil, discret. À refaire. Excellent, rien à redire une référence en restaurant japonais  Accueil sympa, cuisson et assaisonnement parfaits Excellent. Meilleur gyoza. Ma référence en terme de Japonais. Acceuil sympa, ambiamce relax. Un bon moment Nétant jamais allée au Japon, je ne peux me prononcer sur le côté authentique de la cuisine et des boissons servies. Mais lendroit nous plait à chaque visite. Mieux vaut réserver, ce nest pas grand, même avec létage. Le service est rapide, on peut voir les cuisiniers sactiver vers lentrée. Nous aimons le principe des multiples plats à partager. Cest joliment présenté, contenu et contenant, et cest plutôt bon. Jai certes trouvé que le plat de nouilles sautées au poulet que javais pris ensuite était très généreux en nouilles, un peu moins en garniture. Jaime bien les glaces proposées en dessert, aux parfums étranges, haricot rouge, sésame noir...Un grand choix de saké et de liqueurs à base de yusu. Certains nous ont plu, dautre moins, mais cest histoire de goûts. La carte a au moins le mérite par son large choix de nous offrir la découverte. Petit japonais très sympa  La décoration très dépaysante nous met tout de suite dans le bain  Nourriture très savoureuse Ne vous attendez pas à trouver des sushis dans cet excellent restaurant japonais. En revanche les plats typiques sont excellents. Le personnel est très accueillant. Pensez à réserver, le restaurant sur 2 étages est souvent plein. Les nouilles sauté de saveur chinois, cest pas un vrai restaurant japonais. Les ravioles congélés. préférable un restaurant japonais à volonté. Cadre très sympathique, personnel à lécoute, souriant et disponible, les plats étaient savoureux et très bien présentés. Nous avons commandés une dizaine de plats que nous avons partagé pour la découverte et nous avons tout adoré. Rapport qualité-prix parfait. Nous avons tout adoré.. Lambiance, laccueil et bien sur chacun des plats Ce resto est au top du top Hugh Aucune différences entre les plats. Aucune saveur. Les raviolis au légumes. Les crevettes passent encore. Sinon aucun intérêt de venir. Bière pas fraîche. Personnels sympas mais sans plus. Ne se distingue en rien des chaînes asiatiques, donc préférer un restaurant à volonté pour la quantités et le choix. Et c est tout. Resto plutôt cher. Je ne reviendrai pas. Dommage. Pourrir restaurant japonais où lon vis sert de vrais plats japonais plus appétissant les uns que les autres. Accueil très souriant, très polie, tout dans la délicatesse japonaise. Tout y fait pour voyager, les serveuse et cuisiniers qui parlent japonais, lambiance avec une décoration digne des films ou mangas japonais. Un voyage au pays du soleil levant. La nourriture est excellente et le prix nettement abordable. A faire. Le nom du restaurant est japonais, la décoration ressemble à une Izakaya auberge japonaise, le personnel pourrait être japonais mais parle uniquement français et chinois..., les noms des plats sur le menu sont en japonais avec quelques erreurs révélatrices..., mais la nourriture elle-même nest pas, mais alors pas du tout japonaise  Jai passé plusieurs mois au Japon, et fréquenté des restaurants japonais authentiques à Paris Rue sainte Anne, entre autres, jai donc un peu de recul... Soyons clairs, je nai rien contre le fait que des chinois un certain M. FENG en loccurence, gèrent un restaurant japonais, mais quils le fassent correctement, alors  Il y a les fake news, ceci est un fake resto japonais. La qualité des ingrédients est globalement faible par exemple des crevettes pâlotes, caoutchouteuses et sans saveur, la mise oeuvre des recettes, certes vaguement asiatique gingembre, sauce soja.., ne parvient absolument pas à reproduire des saveurs typiquement japonaises. Même la bière japonaise Kirin Ichiban est en fait brassée en Allemagne ce nest certes pas la faute du propriétaire du restaurant même sil existe une version export produite au Japon et disponible en Europe, mais ...plus chère, mais cela ajoute à limpression que tout est bidon ici. En bref, amateurs dauthenticité, passez votre chemin  Signé un amoureux de la cuisine japonaise, bien dégouté, et amer de sêtre fait avoir. Pas de sushis, mais des plats japonais basiques et bien réalisés  des petites salades, des raviolis, des donburis, des ramen Et un joli décor qui donne limpression dêtre au Japon. Une petite adresse sympa qui ne fait pas mal au portefeuille. Bonne découverte  Plats remplis de saveur. Je recommande ce restaurant - Les desserts sont à découvrir Bon accueil, belle décoration, et les plats sont savoureux. Lespace cuisine est visible, ce qui est très appréciable. Je recommande Respect dune cuisine méconnue. En effet les japonais mange rarement des soushis. Rare sont à lyon les bons soushis à lyon, mais çà éxiste miko. Bon service, bonne présentation, commerçant. bONNE ADRESSE Très bon accueil On a choisi plusieurs petites entrées et nouilles en repas principal. Dessert tiramisu au thé vert fut une belle surprise Allez y les yeux fermés Jolie déco qui ma dépaysée dans ce tout petit restaurant coloré. Le service est rapide et un peu distant, mais je crois que cest culturel  Des plats excellents et un prix raisonnable  Une adresse à recommander Petite adresse sans chichi et authentique. Des belles assiettes savoureuses et du japon plein les papilles  Je reviendrai avec plaisir pour goûter cette fois la carte des Saké, qui avait lair prometteuse. Cet endroit est tellement mignon. Un vrai restaurant japonais. Les plats sont typiques, bons et accessibles. Mention spéciale pour les pâtes qui bougent je nen dirais pas plus. Laccueil est charmant. Ils ont une superbe collection de saké. Allez-y les yeux fermés, dépaysement assuré. Pensez à réserver Nous avons dîné à 2 un samedi soir chez Kuma, alléchés par la carte et notre envie de retrouver des saveurs japonaises. Les  entrées  étaient vraiment bonnes  salade de poulpe 10,5, petite mais goûteuse, gyozas aux légumes vraiment top 5,5 les 5 et takoyakis délicieux 5 les 3 Par contre le reste a été décevant yakisoba ressemblant à des nouilles sautées chinoises avec les mêmes légumes coupés super fin, très collantes et sans saveur. Le donburi à languille était moyen et à 18 Addition salée donc, il faut mieux privilégier plusieurs entrées mais nous navons pas testé les ramens Quel regal, une cuisine vrai cuisine japonaise de tout les jours. Ici pas de brochettes yakitori ou divers sushis mais des plats copieux tous aussi savoureux et variés. A faire et refaire Tout dabord, la déco est génial, très colorée et habillée. On se voit déjà parti au Japon en entrant dans ce restaurant. Le service est aimable et rapide. La carte, change des habituels SushiMaki. Ici, cest plutôt Ramen, Yakisoba, Unagido, entre autre. Les plats sont délicieux, le prix est modeste pour un japonnais. Une belle découverte, jai adoré. Je reviendrais sans hésiter Enfin un bon restaurant japonais  Cadre très agréable et chaleureux, le personnel est agréable et la cuisine est super bonne, déjà 2 fois que je vais y dîner, cela va continuer  je recommande fortement ce restaurant Bistrot japonais très agréable, et même plus. Cétait la deuxième fois que je venais et je nai pas encore été déçu. En effet, ma première visite mavait fait goûter lexcellent Udon, fondant à souhait dans la bouche, gluant sans être collant. Je suis revenu prendre un Yakisoba et pareil, entièrement conquis par sa légèreté en bouche et son goût exquis qui ma empêché de le déguster trop longtemps. Peut être que les portions sont trop petites pour moi  En tout cas je reste sur ma faim tellement je me régale et malgré le prix bien abordable, je prendrais sans hésiter deux assiettes juste par gourmandise, alors quelles sont bien suffisantes pour mes proches  Pour conclure, les dessert à base de thé vert macha achevaient en beauté ce repas dans un cadre très fidèle au Japon, dont je suis un grand fan  À essayer durgence Ce qui est extraordinaire cest que le chef est chinois mais il a bien compris lexcellence de la cuisine japonaise Une bonne initiation au saké car trop de monde penses à tord que cest fort... Bel endroit où le dépaysement est total Petite taverne japonaise chaleureuse avec une jolie salle à létage. Les plats sont bons et variés, découverte de nouvelles saveurs notamment la bonite séchée un régal  Large choix de saké et de bières japonaise .... En suivant les critiques très élogieuses sur cette établissement nous y sommes rendu pour dîner, Décor très joli très japonisant Personnel accueillant et agréable Apéritif alcool de prune au sucre noir avec thé vert excellent Plats très bon pour ma part jai pris les nouilles aux légumes et pour terminer un très bon tiramisu matcha Très bon rapport qualitéprix Restaurant très agréable à refaire les yeux fermés Je vous le conseille vivement Très bon restaurant que je recommande vivement  Pour linstant le meilleur japonais de Lyon daprès moi, les gyozas étaient excellents, le gyudon était délicieux aussi Très bon restau Japonais, plats vraiment typiques servis dans de la très jolie vaisselle japonaise, cadre sympa, surtout la salle à létage, très jolie ambiance. Nous avons même aimé le toffu Hiyayakko alors que ce nest pas trop notre truc. Les gyoza grillés aux légumes sont excellents, les Ebi Fry crevettes panées, les nouilles topissimes Yakisoba poulet, poulet udon, curry ramen. Pour les desserts encore une immersion complète au Japon, avec le Dorayaki sorte de pancake farce haricot rouge ou le match gourmand, le tiramisu au thé vert excellent  On a hate dy retourner pour gouter dautres plats, grands choix également de boissons locales, avec la ramune limonade du Japon, les whiskys, pierre, cocktail... BRAVO Pour le moment, cest le meilleur japonais que jai pu manger à Lyon. 100 authentique et pas un restaurant à sushi. On y reviendra souvent. Très bon bistrot japonais . Beaucoup de choix niveau saké . Resto idéale pour déguster de lalcool japonais. Très bon restaurant japonais Accueil agréable, une décoration typique et les plats sont de qualité Une valeur sûre Le cadre est sympa. Ça ressemble assez à un izakaya Les nouilles du ramen sont trop molles. Le service est rapide et bien Soirée avec un ami. Jadore ce restaurant. Laccueil est comme toujours très agréable. Les plats sont simples mais très bon Rien à redire si ce nest allez y Vraiment marre dêtre déçue par lavarice des ingrédients, un assaisonnement trop juste sans prise de risques et le doute sur les produits. Après repère sur le menu nouilles soba au sarrasin, proposées au choix avec poulet, porc ou légumes. Je prends loption légumes  34 tranches de carottes coupées à la mandoline, autant dire ultra fines, et la même chose pour la courgette voilà un plat dit de nouilles aux légumes  ce qui est sur cest que le chef ne ma pas régalée. En revanche, Il a maîtrisé son coût de revient sur lassiette. Je doute de la nouille soba habituellement de couleur grise qui avait plus des allures de nouilles aux oeufs mais peut être du au soja et sa couleur  un mot sur lassaisonnement dans la même trajectoire, limite. Et Oui, déçue déçue déçue, car jadore la nourriture japonaise. Trop gourmande, je suis restée sur ma Finfaim Il est suffisamment rare de ne pas les voir à la carte des restaurants japonais communs quil devient important de signaler ici ramen, salade dalgue et thé au yuzu. Les ramens sont servies en quantités copieuses qui suffiront à elles-seules un appétit moyen voire plus. Il vaut mieux réserver car les tables ne sont pas si nombreuses. Le service est correct mais pourrait savérer long à un moment de forte affluence. En tout cas la qualité de ce qui nous a été servi était telle quelle invite à revenir. Après plusieurs visites chez Cuma, je reste toujours aussi satisfaite  Les plats proposés sont vraiment très bons, bien assaisonnés, et en quantité suffisante. Les entrées sont excellentes je vous conseille les gyoza, ils sont excellents, et les plats sont très bien exécutés. Le choix peu semblé restreint, mais il apparaît finalement suffisant. Le restaurant propose des plats de viande, mais aussi des plats végétariens  un véritable avantage pour ceux qui ne consomment pas de viande ou de poisson. Mention rapide sur le personnel, toujours très sympathique et souriant  Nous avons été accueillis à bras ouverts avec notre petit chien. Je recommande Restaurant testé samedi soir suite aux avis très favorables et là quelle déception, nous sommes je ne sais ou mais en aucun cas au Japon. A 4 nous avons pu goûter quelques plats qui font lunanimité, ce nest pas terrible et pas du tout japonais, même la cuisson du riz est ratée Nous étions au Japon au printemps dernier et quand on lit ces éloges comme à Tokyo ou pire Kyoto, sil vous plait, sortez un peu avant dinciter les autres daller mal manger. Rarement nous avons été aussi déçu dun restaurant et surtout des différents commentaires de Trip. Un petit restaurant japonais qui vaut le détour. Un accueil agréable, dans un cadre typique japonais. Service rapide et impeccable. En entrée Tamago Yaki une omelette japonaise pour 3 prix très correct pour un goût vraiment bon. En plat un tchachyu-ramen pour 12, vraiment quel délice. Très bon et copieux. En dessert Doryaki pour 5, un sandwich de pancake avec au centre une pâte de haricots rouge sucré et à côté un boule de glace à lharicot rouge avec une framboise. Je me suis régalé. Au final un cadre très agréable service et intérieur, avec un repas qui est vraiment bon. Je le recommande. Jai gouté leurs ramens et je nest pas pu les finir car trop pour moi elle était bonnes mais jai pu en gouté de meilleurs les gyoza était très bons seul mon tonkatsu ma déçu sinon la salle est petite mais agréable le personnel agréable nous a même offert des petits origami jai passé une bonne soirée PARFAIT une équipe extraordinaire une carte des boissons de qualité des thés alcools des boissons originales , la décoration insolite traditionnelle nous nous croyons transporté à Kyoto et surtout un service impeccable un lieu propre joli attrayant, un service dune immmense gentillesse efficace et la cuisine un délice succulente extraordinaire pour qui connait la cuisine japanaise elle sort des sentiers battus ...sushis ou yakitori courez découvrir ce lieu merveilleux et magique pensez à réserver Vrai coup de cœur de Lyon Vrai nourriture japonaise. Quantité, goût même le décor faisait penser au Japon. Fin gourmet de la cuisine asiatique franchement je recommande Le Japon à Lyon Venant dAnnecy nous nous sommes arrêté dans ce petit resto qui paye pas de mine de lextérieur mais cest une vraie perle du japon à Lyon. Ça vaut presque les 3 heures de route rien que pour ça. Nous avons retrouvé les même saveurs quà Tokyo Le tout avec des tarifs très abordable. Un voyage à lautre bout du monde au coeur de Lyon Nous y avons été sur conseils dun ami et nous navons pas été déçue. Malgré le monde nous avons été correctement servis. Serveuses avec le sourire ce qui est de plus en plus rare de nos jours. Les ramens sont vraiment excellent biens meilleurs que rue sainte Anne à Paris. Lambiance du restaurant est très agréable et la nourriture est bonne. Jai trouvé le rapport qualité prix un peu cher et le service très long. Peut saméliorer sur certains points. Je recommande leur ramen Une belle adresse à partager Cadre trés agréable Belle carte Tchachyu-ramen excellent essayer le Unagi-don Desserts au top tiramisu au thé vert et le dorayaki Lambiance est très sympa, les serveurs très sympathiques.Le service est rapide. Les plats dégustés sont très bons même si nous les avions trouvé pas suffisamment copieux pour nous nous étions 3 que vous pourrez voir sur les photos. De passage à Lyon avec mon mari et mes enfants, jai eu lexcellente surprise de découvrir ce petit bistrot japonais. Enfin un japonais qui ne fait pas de sushis mais qui propose de la nourriture typique japonaise à des prix des plus abordable. Leur menu du jour à 13  avec un plat principal, du riz, une soupe miso, une bouchée de crudités, cest le rêve. Nous avons choisi chacun un plat différent, tous étaient excellent. Sans parler de leurs desserts originaux et très bon. Leur tiramisu au matcha était délicieux et dune légèreté incroyable. Même si nous avons un très bon japonais à Annecy, je regrette que nous ayons pas un bistrot comme celui-ci avec des prix plus abordable. Je nhésiterai pas à y retourner lors dun prochain passage à Lyon. Tout est très bon, le décor est très agréable. Nous avons pris des ramens, et des gyozas ainsi que des takoyakis. Très bon rapport qualitéprix. Nous reviendrons Tout est délicieux. Le restaurant est très accueillant, les serveuses vraiment attentionnées, les plats rapidement servis sont de très grande qualité. Pour avoir testé un autre Resto du même genre sur lyon, ca na rien à voir et celui là est vraiment dun niveau au-dessus. Nhésitez plus et foncez à Kuma Izakaya Enfin un vraie restaurant Japonnais. Les seul à ma connaissance était ceux rue St Anne à Paris un peu loin je suis de Grenoble. Ici ont retrouve la vraie nourriture du Japon et pas seulement des sushis dailleurs yen à pas et cest tant mieux. Place aux Ramens, Hiyayako, Gyoza et autre joyeuseté du pays du soleil levant. En plus maintenant ont y trouve même des Takoyaki  Alors après gustativement  je trouve que les Takoyaki manque de lélément principal à savoir du poulpe sur les 3 jai senti un seul morceaux de poulpe, les Korokkes manquaient de tenue mais je peu pas comparer jen ai pas manger au japon encore. Le tiramisu manque de thé vert à mon gouts mais là cest une histoire de gouts, mais il est vraiment très bon quand même. Par contre le ramen au curry est délicieux, juste assez fort pour moi, et servie avec du curry à coté pour ceux qui en voudrais plus. Les gyosas au légume sont parfait. Je reviendrais avec grand plaisir dans votre restaurant, même si je doit faire 1h de route  cest rien comparer au 13h de vol pour le Japon après tout  Et bien sur, joublie lambiance, qui est vraiment très proche des Izayaka Japonnais et le personnel très courtois et gentil. Un petit restaurant où on est très bien reçu, une carte avec peu de choix, ce qui est plutôt un signe de qualité. Jai très bien mangé et ai été vite servie Je reviendrai Ce restaurant est vraiment super pour qui voudrait un petit peu dévasion et se retrouver comme au Japon Les plats sont bon et authentiques. Les serveurs et serveuses sont disponibles et accueillants. Les prix sont un peu élevés. Très très bon restaurant japonais. Avec des vrais plats et non pas des sushis  Jai testé le kinpira, le tonkatsu et le Macha gourmand. Jai adoré. Rapport qualité prix impeccable. Amateur de cuisine exotique et surtout japonaise, jai découvert ce peut-il bistro japonais en passant devant et quelle belle surprise Petit endroit cosy tenu par 2 jeunes japonais remplis de gentillesse et dhumilité. Très bon Ramen et très bien servis. Et enfin des bières japonaises différentes des traditionnelles kirin et asahi. Une belle collection de wisky et saké japonais. Très bon rapport qualité prix. Je recommande sans hésitation. Un restaurant super agréable des plats typiques très bien présenter et très très bon. Le service est très bien ainsi que le cadre. J y retournerais Après un voyage au Japon javais envie de retrouver un bon ramen en France. Pari réussi au Kuma. Lambiance de la salle comme le goût du ramen mont ramené au Japon. Un bon ramen au sésame grillé. Un accueil gentil et chaleureux. Un petit goût de Japon en plein lyon. voilà longtemps que je navais pas mangé un aussi bon Japonnais  Le cadre est simple, les employés vraiment serviables  Je recommande Un japonais que je recommande  Atmosphère cosy, serveurs très aimables et plats succulents  Le tiramisu au thé vert est original Un endroit super sympathique, très chaleureux et convivial. Les ramens sont de loin les meilleurs que jai pu manger, et les petites entrées sont délicieuses Je vous conseille les gyoza. Les prix sont plus que raisonnables pour la qualité. Découvert un peu par hasard treap adviser aidant... nous avons été très content. Les plats étaient bon. La salade de poulpe très bien assaisonné. On reviendra Je suis allée deux fois au Japon et à chacun de mes retours ça a été un déchirement de ne pas retrouver en France les bons petits plats que jai pu déguster là-bas. Maintenant je sais que je retournerai au Kuma Izakaya pour bien manger japonais et à un prix tout à faire décent  Nous avons goûté le ramen au porc, le poulet teriyaki et les gyozas. Puis en dessert, de la glace au sésame noir. Tout nous a régalé  Le seul petit bémol pourrait être que les portions sont assez modestes. Les gros mangeurs vont peut-être devoir plus commander que les autres. Quant au service, il a été très agréable et discret. Un cadre typique et dépaysant, une carte sans les sempiternels sushis  makis mais proposant des plats bien plus traditionnels. Service discret et efficace, bon rapport qualité  prix. Bref, rien à redire... sauf que les gyozas aux légumes sont juste fabuleux La cuisine est fine et le service bien fait. Le tiramisu au thé vert est a essayer Merci pour votre accueil La discretion des hôtes est à la hauteur de ce que vous allez manger. Tout est bon, la carte raffinée, la selection de whisky parfaite. Cest parfait pour un déjeuner daffaire, pour un déjeuner entre collègue, un moment en amoureux  Merci Super petit resto japonais, petite carte et produits frais. Tout était très bon, les crevettes panées excellentes, les udons super bonnes, mais le must cest les gyoza aux légumes,un régal. Bon choix de bières et whisky japonais. les plats sont vraiment savoureux et on se régale pour pas très cher  perso je suis fan des koroke miam Bistro japonais top. Bon rapport qualité prix. Je conseille pour ceux qui veulent changer des traditionnels sushis. Jai juste pris un plat, il était excellent poulet-udon  je ne met pas la note naxi, car la biere locale est un peu cher en rapport qualité-prix et dessert décevant mais cest vrai que ce nest pas ce quon attend dun radio japonais forcément Un resto japonais sans sushis. Parfait, les plats sont authentiques, on est au Japon Les clients sont la pour ça, on entend parler du Japon à chaque table Nous navions en fait jamais mangé japonais  Les plats sont originaux, raffinés, Accueil très chaleureux et très sympathique, dans un petit restaurant typique et très mignon. Mélange de saveurs exceptionnel Des petits prix à refaire sans hésitation Ps et en plus nous avons découvert le véritable saké Dîner partagé avec ma famille dans un cadre très sympa. On avait pris plusieurs entrées à se partager et jai pris comme plat des ramen au curry cétait super bon. Et le dessert était un assortiment au thé vert un régal  je recommande fortement   De plus une des personnes qui était avec moi a vécu plusieurs années au Japon et à totalement validé. Enfin de la vraie nourriture japonaise  Vraiment délicieux. Très bien pour les déjeuners professionnels, en couple ou entre amis. Je recommande Pas de soucis on est au Japon Délicieux plats japonais, carte de Sakés impressionnante. Rapport qualité prix parfait Le cuisinier dorigine Chinoise a étudié les arts culinaires au Japon. Que du bon... Service aussi par des Chinoises très bien. On va revenir. Un japonais où on peut manger des plats délicieux. On sest régalé étant tous les deux fans de ces mets. Le service est rapide et très accueillant. Un japonais sans sushis et sans Maki mais en revanche de vraies saveurs dépaysantes Parfait  Accueil sympathique malgré un restaurant bondé et une arrivée sans réservation. Deuxième fois dans ce restaurant et certainement pas la dernière  Les plats sont savoureux et les prix correct, les desserts comme le tiramisu au thé vert Matcha sont très bons aussi. Le cadre est sympa et le service parfait  Je recommande vivement pour un petit voyage au Japon le temps dun repas. Cuisine japonaise, vous ne trouverez pas de sushis, makis... mais plutôt certains plats plus traditionnels des bistrots japonais bol de riz surmonté de viande ou poisson, nouilles...manque certaines choses comme des fritures de légumes. Cest très bon et frais. Les animateurs sont chinois, notamment le chef. Ce qui pourrait casser un peu le mythe mais ce dernier à passé 4 ans au côté de vrais chef japonais...ne voit on pas dexcellent italien tenu pas des français  Laissez vous tenter... Vous ne trouverez pas de sushis comme les banales restaurants japonais.. laissons place à la vraie cuisine où tous les plats sont fins et bien assaisonnes.. un vrai régal. Les prix sont vraiment abordables. Allez y sans regret vous y retournerez a coup sûr Un vrai délice pour les papilles. Tout est de qualité y compris les couverts. Il ne me reste plus quà apprendre le japonais pour pouvoir choisir mon saké parmi la multitude proposés. Cest la deuxième fois que je viens dîner dans ce restaurant de la rue Masséna. Excellent rapport qualité prix, encore pas trop fréquenté, une adresse à ne pas trop vite ébruiter  Vaut les bons restaurants de la rue Sainte Anne à Paris. et copieux contrairement a ce quune personne nous a signifié quand nous regardions la carte dans la rue. Cuisine intéressante qui permet de découvrir la cuisine japonaise en dehors des sushis. Prix raisonnables. Une bonne adresse Cuisine japonaise authentique Service parfait produits frais Prix contenu et bon choix de saké  je conseille les shosu  Très semblable aux bonnes auberges simples que lon peut trouver au Japon À recommander sans modération Des spécialités japonaises à tomber. Jamais eu en France une telle qualité. Adepte du poisson cru passez votre chemin ici il ny en a pas On a fait woaw en voyant arriver les plats et alors en les goutant....un regal. Maintenant on sait qu il n y a pas que les sushis dans la cuisine japonaise. Bien que jaime beaucoup les sushis jai découvert les plats traditionnels japonais. Les plats ont été très bien cuisinés et excellents. Tout petit resto, même sil y a une salle à létage, mais cuisine très fine et savoureuse. mention très bien pour la salade de poulpe  Beaucoup de fraicheur dans les plats, présentés comme il se doit avec un soin et une méticulosité toute japonaise. Loin des pseudo restos japonais de sushis...Tant mieux  Prix tout à fait corrects. Réservation plus que conseillée. Nous découvrons ce petit restaurant japonais assez sympa, et si nous prenons des plats assez classiques, nous découvrons de nouvelles saveurs au niveau des desserts avec de très bonnes glaces ou les fameux dorayakis cf film Les délices de Tokyo qui nous ont réjouits  Laccueil était très sympathique, et pour une vingtaine deuros par personne, nous avons très bien mangé et suffisamment  Une soupe, un plat et un dessert chacun. En bref, très bonne adresse pour sévader Décoration sympathique, nourriture excellente , accueil parfait . Je ne suis jamais allé au Japon mais quel bonheur de venir ici déguster ces fabuleux plats , simples mais délicieux . Un rapport qualité prix Bistrot , restaurant , bar .... Bref ma cantine . Nous sommes allés manger avec ma conjointe dans petit bistrot japonais.Le repas était parfait pour un voyage dans ce pays de tradition culinaire très riche. Nous reviendrons très bientôt Simple, accueillant. Des plats savoureux et simples mais éloignés des habituels sushis et sashimis. Je recommande les 3 verres de sake. Le sunomono est un régal. Avec le korokke, mieux vaut prendre un complément. la salle est petite mais salle en haut. Ouvert le dimanche soir. Repas en amoureux un samedi soir qui sest déroulé a merveille. Le cadre rappel vraiment les échoppes Japonaises. Les plats à la carte sont très bon et en plus à des prix abordables. Aucun point négatif à remonter. Très bonne découverte dans ce bistrot japonais  Ici pas de sushis  Cie, mais des plats typiques succulents beignets de crevettes fondants, raviolis savoureux, légumes crus très bien assaisonnés, servis avec sourire et convivialité dans un cadre sympa et pour des prix corrects. Et pour une fois, aucune déception sur le dessert  un tiramisu au matcha truffé de haricots rouges, une merveille de douceur et de goût. Vraiment ravie de ce lieu, et en bonus cest tout près de la gare, parfait pour une escale Cela fait 2 fois que nous nous rendons dans ce bistro japonais et à chaque fois nous nous régalons Les plats sont très bien présentés et surtout ils ont un goût absolument divins... Je suis allée au Kuma plusieurs fois et je ne men lasse pas Des Soba et Udon du tonerre, un the au yuzu dont le souvenir me fait saliver et un tiramisu au thé vert à tomber Je vous conseille le Kuma, on y est bien reçu et lambiance est calme et reposante Kuma Izakaya est le genre de restaurant où vous ne pouvez pas vous tromper en choisissant vos plats. Tout est bon, les prix sont corrects et le cadre sympa. Si vous souhaitez tester un restaurant japonais différent, cest ici quil faut aller. Je regrette toutefois les desserts médiocres et pas tentants, comme dhabitude dans les restaurants asiatiques. Je vous recommande le udon légumes  Attention  pas de suschi à la carte Quand on parle de cuisine japonais on imagine que les sushis, et bien détrompez vous. Ce restaurant vous fait découvrir une autre partie de la gastronomie japonaise, avec des saveurs extrêmement bien équilibrés, une justesse des assaisonnement et du gout qui explose en bouche à chaque bouchée. Laccueil est extrêmement chaleureux et poli. Je recommande vivement cette invitation au voyage. Très bon accueil dans ce petit restaurant qui propose des plats typiquement japonais oubliez les sushis et les brochettes. On se régale, quel que soit le plat choisi, avec des saveurs étonnantes glace au haricot rouge ou tofu mariné avec de lhuile et du gingembre. A faire et à refaire Belle découverte  Super resto très bon  Accueil aimable  A recommander . il vaut mieux réserver si possible car la salle est petite Un petit restaurant japonais sympathique. Les plats sont de bonne qualité et copieux. Un grand choix de boissons typiques. Un accueil agréable. Que demander de plus On se croirait rue saint Anne à Paris Enfin un vrai japonais à Lyon Tout y est typique, des ramens au curry japonais aux gyosas vapeur ou grillés Un endroit qui change des restaurants japonais à sushis aseptisés  À tester Pour ne pas payer de mine a lextérieur, lintérieur est tout autre. Cest là que ça se passe. Un lieu plutôt agréable ou lon est pas serré comme des sardines. Un menu simple, efficace et bon avec au choix des menus ou des plats ici pas de sushis et autres sashimis cuisinés, qui semblent frais et qui sont plutôt bon. Le tout accompagné dun prix plutôt léger, et dun personnel gracieux et très avenant. Enfin plein de choix de bières, whisky, saké, plus quil nen faudra pour un repas  Un vrai bon restau comme on les aime, sans prise de tête, calme, et à prix plutôt tout doux. Convient très bien aussi en famille avec enfant crevettes, riz, .... Pour etre parti au Japon cet été et fréquenter régulièrement notamment les resto japonais de Paris je peux assurer de la qualité de Kuma Izakaya. Les gyozas sont délicieux et la sauce qui les accompagne à base dhuile de sésame est vraiment succulente. Les ramens sont également de très bonne qualité. Le service est également au rendez vous même lorsque lon arrive en retard sur lhoraire prévu après avoir prévenu, cest la moindre des politesses et le cadre est très très sympa. Bref encore une adresse à recommander sur Lyon. Les nouilles udon et ramen sont très délicieuses  Les serveurs sont très sympas et le service est très agréable Cuisine traditionnelle japonaise. Ce petit restaurant sert des ramens très appréciables. Jai passé un excellent moment. Un gros plus pour loriginalité des desserts Je connais bien la cuisine Japonaise de tous niveaux. Je mattendais à trouver les classiques de la cuisine populaire Japonaise et jai commandé un tonkatsu et mon mari un tempura de crevettes Le tempura ressemblait à des crevettes panés surgelées, sans goût rien à voir avec des tempuras. Idem pour le tonkatsu dur comme du cuir, même pas fait avec la panure quil faut. Limpression est que tout cela est du surgelé acheté chez un distributeur Même le riz était mal cuit, ce qui est un comble pour un restaurant dit Japonais. En plus ce nétait pas donné vu ce qui était dans lassiette et apparemment aucun des personnels ne parle Japonais ce qui peut expliquer ce désastre. Bref je me suis sentie trompée de A à Z. Heureux de découvrir un nouveau restaurant japonais traditionnel à Lyon, nous avons rapidement déchanté. Ce qui était vendu comme un tenpura était en fait un ebi furai ce nest pas la même chose , le tonkatsu était trop dur et sa panure nétait pas très croustillante, le riz manquait un peu de cuisson, le miso semblait instantané... Le tout avec des proportions peu généreuses pour le prix. Seuls les gyozas étaient bons, même sils ne semblaient pas fait maison. Il y avait également quelques erreurs de japonais dans le menu par ex. un curry ne sécrit pas kari mais kare en katakana et les baguettes nétaient pas japonaises extrémités trop épaisses, montrant que ce restaurant nest pas très authentique... ce qui aurait pu être acceptable si le repas avait été à la hauteur. Heureusement, le service était plutôt agréable et le cadre assez proche dun petit izakaya de quartier. Un non-initié peut éventuellement apprécier, mais si lon sy connait un peu en cuisine japonaise, ce nest pas une adresse à retenir. Un petit dîner entre copine pour tester ce restaurant. Nous avons été très bien accueillit, service rapide. le tiramisus au thé vert est en effet une tuerie, je vous le recommande. Pour le cote qualitéprix ils sont très raisonnables. Bref bon bistro japonais. je pense que jy retournerai rapidement. Le restaurant ne paie pas de mine de lextérieur mais il est très chaleureux et bien décoré à lintérieur  Je me suis régalée en prenant les gyozas grillés qui sont vraiment très bons. Jai eu loccasion de tester beaucoup de gyozas dans divers restaurants et je peux vous dire quils sont fins et la garniture est vraiment très bonne. Et les yakisobas sont elles aussi très bonnes  Jai passé un agréable moment dans ce restaurant charmant et le service était lui aussi très bien. Si vous aimez les restaurants japonais sans sushi ni makis nhésitez plus et venez au Kuma Le meilleur resto japonais de Lyon, OUI , le meilleur Tout est fait maison, avec le service très sympatique. Pas loin de la gare Part-dieu. Très bon rapport qualité-prix. Malheureusement, jai pas pu de goûter les sakés comme je suis allergique. Excellente cuisine japonaise familiale. Accueil tres agreable. Super adresse de bistrot japonais. Chez kuma, on est loin du piege a touriste ou autre bars a sushis .Ici cest qualite et sourire Nous venons de découvrir ce petit bistrot typique japonais, et nous en sommes fort satisfait. La déco, la carte, la cuisine et le service ont lair directement importé de Tokyo. Lambiance est très conviviale, et le service impeccable et chaleureux. Il y a un vaste choix de saké  à découvrir impérativement et les plats sont très savoureux tout en gardant une simplicité typiquement japonaise. Nous vous conseillons les plats suivants  tonkatsu, yakisoba, yakiudon, poulet teriyaki. Enfin un vrai japonais Une découverte magnifique que ce bar japonais au 93 rue Masséna. Un accueil charmant, un endroit très reposant et surtout une cuisine sublime, tout en saveurs et finesse. On est loin de ce que lon a lhabitude de manger dans cette catégorie de cuisine. Le tiramisu au thé vert est un délice Bonne cuisine japonaise à prix doux,service sympathiqueJai particulièrement adoré lassaisonnement de ma salade Il ny rien à redire sur la qualité des plats. Les Gyoaz grillés sont divins, les ramens également même sil manque un peu de viande dans le Tchachyu Ramen. Si vous voulez découvrire autre chose que les sushi dans un japonais, je vous conseil fortement de faire un tour la bas. un ami ma fait découvrir ce restaurant hier..qualité prix irréprochable..service rapide et en plus la patronne est très aimable .je le recommande et je reviendrait a faire connaitre.. Nous avons passé un bon moment. Le service était rapide et très souriant. Je ne suis pas une grande fan des spécialités japonaises, mais jai bien apprécié le repas, la nourriture était de qualité. Je recommande Rare de trouver des cantines de ramen japonaises, celle-ci est excellente  tout est frais et préparé avec précision, les prix sont au plancher et les ramen excellents, le forfait dégustation de 3 sakés différents est généreusement servi et documenté. La serveuse est adorable. Bref, un endroit à découvrir absolument Bon restaurant japonais, bon accueil, décoration qui rappelle le Japon avec la possibilité de manger au comptoir. Plats de qualité et à un bon prix  tempura, gyoza, ramen... Le bouillon du ramen n est pas le meilleur que nous ayons mangé car un peu fade ce soir là mais nous retenterons l aventure sans aucun doute. Mention plus pour les produits au matcha. Je suis venu manger dans ce restaurant japonais quatre fois depuis sa récente ouverture et je nai jamais été déçu. Jaime faire découvrir ce resto à mon entourage qui sort toujours ravi de son passage. Faut dire que niveau qualité-prix on ne fait guère mieux à Lyon. Et puis pour ceux qui souhaitent une nourriture japonaise plus familiale, cest lendroit rêvé. Pour couronner le tout, laccueil est chaleureux, on prend plaisir à rencontrer les propriétaires qui mettent toute leur énergie dans leur restaurant avec passion et tendresse. Un lieu calme et agréable où le poulet teriyaki est divin... Lundi midi de Pâques, nous nous aventurons dans ce petit bistrot japonais et quelle bonne idée  Laccueil est charmant et les mets ultra-frais et authentiques. Pas de sushis ni de makis mais plein dautres bonnes choses  Le sunomono salade dalgues et de concombre est délicieux ainsi que les plats à base de nouilles sautées, les gyoza, les bouchées de viandes poulet, porc pané. Le cadre est mignon comme tout, mais attention on est bien dans une ambiance bistrot, petites tables et tabourets en bois et rien de superflu. Cest petit, pensez à réserver, dailleurs on garderait bien cette adresse pour soi Jai tout de suite retrouvé lambiance du Japon en entrant dans ce petit restaurant. A larrivée des plats aucune déception. Les goûts sont prononcés, sésame, gingembre, katsuobushi, miso et jen oublie. Toutes les saveurs font immédiatement voyager. Je recommande les yeux fermés, je nai quune envie cest dy retourner tester tous les plats. Une très bonne surprise au coeur du 6e arrondissement de Lyon  Lextérieur nest pas très attirant mais une fois à lintérieur, lambiance japonaise cosy a son charme. Les plats sont excellents. De très bonnes saveurs à des prix très raisonnables. Une carte avec du choix et des formules intéressantes y compris les desserts. Le lieu est assez petit. On sy sent comme à la maison. Laccueil est chaleureux, souriant et arrangeant  Une très bonne adresse comme il en manque. Jy retournerai sans hésitation  Petit bonus  le tiramisu au Thé vert, une TUERIE  la vaisselle en céramique Coup de fatigue ce soir, pas le courage de cuisiner et pourtant... grosse envie de me régaler. Après deux minutes de réflexion je me souviens avoir remarqué rue Massena une petite enseigne inconnue signe douverture récente étant donné que jhabite à deux pas qui a le bon goût dêtre rédigée en japonais, ce qui pour un restaurant japonais est en général plutôt bon signe. Que dire sinon que jai trouvé ma nouvelle cantine  Le service est on ne peut plus agréable et attentionné, et peut-être certains trouveront-ils le cadre légèrement impersonnel mais cest de rigueur pour un Izakaya et vraiment, il émane du personnel et des quelques éléments de décoration une gentillesse perceptible. Je ne fais pas durer plus longtemps le suspense, les plats sont à la hauteur de tout le reste, excellents, et il me serait impossible de ne pas parler de lattention particulière portée à la présentation tout simplement incroyable dans un Izakaya et à ce niveau de prix. Un signe qui ne trompe jamais, le unadon, parfois appelé Unagi don, un plat à base danguille nest véritablement réussi que languille a mariné suffisamment longtemps pour que le goût poissonneux disparaisse et quil ne reste que des bouchées crémeuses et délicatement sucrées. Jai mangé chez Kuma le premier unadon correct que jai trouvé à Lyon après une bonne quinzaine dessais décevants, et il nétait pas simplement correct, il était délicieux. Tout cela suffirait à garantir à lendroit un 55 bien mérité, mais en plus il me faut vous parler du tiramisu au thé absolument délicieux, du thé vert japonais parfaitement authentique sans sucre et mousseux à souhait et pour finir, de la sélection de whisky et digestifs malheureusement trop peu mise en avant mais très soignée. Cest lors dun voyage à Lyon que jai découvert ce très bon restaurant situé près de la gare. Laccueil y est chaleureux et la cuisine de très bonne qualité. Les saveurs sont au rendez-vous tant pour les plats que pour les desserts, ce qui est rare. En effet, jai pris un poulet rôti sauce soja sucrée avec une soupe très goûteuse et pour terminer je me suis laissée tenter par le Tiramisu au thé vert, super bon On sort en ayant mangé équilibré avec des produits de qualité, tout en se faisant plaisir tout ce que jaime Le rapport qualitéprix est donc très satisfaisant. Je prévois dy retourner pour mon prochain séjour et vous le conseille vivement. Tout nouveau restaurant, cest avec plaisir que jai pu découvrir ce bistrot japonais qui a répondu parfaitement à mes attentes Une nourriture excellente, un rapport qualitéprix parfait, et un service fort aimable et souriant Un restaurant à suivre que je vais mempresser de faire découvrir autour de moi Izakaya avec sushi non mais vraiment une cuisine excellente pour un prix raisonnable. On sy sent un peu comme au Japon, cest très calme et le personnel serviable. Il faut absolument y aller si vous êtes à la recherche dnon - cuisine française. Tout simplement génial  La nourriture Je suis venue ici avec ma petite amie, le personnel était très gentil et la nourriture tout à fait idéal. Un adorable petit hôtel japonais déco savoureuse cuisine et service parfait  Je pense que pour le déjeuner avec des prix raisonnables. Il faut réserver une table avant de venir. Le service est très lent, mais les serveurs sont sympathiques et gentils. Vous devriez essayer le riz dalcool, cest bon et le dîner. Jaime le tempura. Cest un hôtel parfait. Javais loccasion de revenir dans cet hôtel à deux reprises. Le prix est plutôt cher, mais cela vaut le coup. Jai séjourné dans cet hôtel 2 fois. Il faut réserver une table avant de venir. Si vous pouvez, essayez donc laisser le riz dalcool dont jai oublié le nom, cest parfait pour vos repas. Ne pas ayant des sushi dans cet endroit, mais les autres repas sont bons, je préfère le tempura. Le prix est plutôt cher, mais cela vaut le coup. Ce nétait pas extraordinaire, mais jai bien aimé lambiance et le service était très sympa. Jai eu le plat de poulet udon soupe et ma sœur a pris lramen, ce qui sest beaucoup mieux passé que le mien."""

ministral8b = MistralAPI(model="ministral-8b-latest")
print(ministral8b.query(f"Résume les avis suivants : '{review1}'"))

### Résumé des avis

#### Points Positifs :
1. **Ambiance et Décoration** : La plupart des avis soulignent l'ambiance typique et la décoration japonaise du restaurant, qui donne l'impression d'être au Japon.
2. **Cuisine** : La cuisine est généralement saluée pour sa qualité, avec des plats variés et savoureux. Les ramens, gyozas, takoyakis, et autres spécialités japonaises sont particulièrement appréciés.
3. **Service** : Le service est décrit comme rapide, efficace, et souvent très sympathique. Les serveurs sont souvent mentionnés pour leur accueil chaleureux et leur disponibilité.
4. **Rapport Qualité-Prix** : La majorité des avis trouvent que le rapport qualité-prix est excellent, avec des plats de qualité pour des prix raisonnables.
5. **Diversité des Plats** : Le restaurant propose une variété de plats japonais traditionnels, allant des ramens aux gyozas, en passant par les desserts au thé vert.

#### Points Négatifs :
1. **Problèmes de Réservation** : Plusieurs avis mentionnent 

In [6]:
ministral8b = MistralAPI(model="ministral-8b-latest")
print(ministral8b.query(f"Résume les avis suivants en ne proposant qu'une conclusion succinte : '{review1}'"))

Kuma Izakaya est un restaurant japonais authentique et savoureux, offrant une cuisine traditionnelle et des plats variés. Les avis sont globalement positifs, soulignant la qualité des plats, le service chaleureux et l'accueil sympathique. Les desserts, notamment le tiramisu au thé vert, sont particulièrement appréciés. La décoration typique japonaise et l'ambiance cosy contribuent à l'expérience culinaire. Les prix sont raisonnables compte tenu de la qualité des plats. Cependant, certains avis mentionnent des problèmes de service ou de qualité des plats, mais dans l'ensemble, Kuma Izakaya est recommandé pour une expérience culinaire japonaise authentique.


In [7]:
ministral8b = MistralAPI(model="ministral-8b-latest")
print(ministral8b.query(f"Résume les avis suivants : '{review1}'", temperature=0))

### Résumé des avis

#### Avis Positifs
- **Ambiance et Décoration**: La plupart des avis soulignent l'ambiance chaleureuse et la décoration typiquement japonaise du restaurant.
- **Cuisine**: Les plats sont généralement décrits comme excellents, savoureux et authentiques. Les ramens, gyozas, takoyakis, et autres spécialités japonaises sont particulièrement appréciés.
- **Service**: Le service est souvent décrit comme rapide, agréable et efficace. Les serveurs sont sympathiques et accueillants.
- **Rapport Qualité-Prix**: La majorité des avis trouvent que le rapport qualité-prix est excellent, avec des prix raisonnables pour la qualité des plats.
- **Desserts**: Les desserts, notamment le tiramisu au thé vert, sont souvent mentionnés comme étant délicieux.
- **Accueil**: L'accueil est généralement chaleureux et sympathique, avec un personnel attentionné et serviable.

#### Avis Négatifs
- **Problèmes de Cuisine**: Certains avis mentionnent des problèmes de qualité de la cuisine, comme 

In [8]:
ministral8b = MistralAPI(model="ministral-8b-latest")
print(ministral8b.query(f"Résume les avis suivants : '{review1}'", temperature=0))

### Résumé des avis

#### Avis Positifs
- **Ambiance et Décoration**: La plupart des avis soulignent l'ambiance chaleureuse et la décoration typiquement japonaise du restaurant.
- **Cuisine**: Les plats sont généralement décrits comme excellents, savoureux et authentiques. Les ramens, gyozas, takoyakis, et autres spécialités japonaises sont particulièrement appréciés.
- **Service**: Le service est souvent décrit comme rapide, agréable et efficace. Les serveurs sont sympathiques et accueillants.
- **Rapport Qualité-Prix**: La majorité des avis trouvent que le rapport qualité-prix est excellent, avec des prix raisonnables pour la qualité des plats.
- **Desserts**: Les desserts, notamment le tiramisu au thé vert, sont souvent mentionnés comme étant délicieux.
- **Accueil**: L'accueil est généralement chaleureux et sympathique, avec un personnel attentionné et serviable.

#### Avis Négatifs
- **Problèmes de Cuisine**: Certains avis mentionnent des problèmes de qualité de la cuisine, comme 

In [10]:
ministral8b = MistralAPI(model="ministral-8b-latest")
print(ministral8b.query(f"Produire une liste de 5 mots clés à partir des avis suivants : '{review1}'", temperature=0.2))

## Résultat non concluant

Voici une liste de 5 mots clés à partir des avis suivants :

1. **Authentique** : Les avis mettent en avant l'authenticité de la cuisine japonaise proposée par le restaurant.
2. **Décor** : La décoration typique et épurée du restaurant est souvent mentionnée comme un point positif.
3. **Service** : Le service est décrit comme rapide, agréable et efficace dans la plupart des avis.
4. **Plats** : Les plats sont souvent qualifiés de délicieux, savoureux et bien présentés.
5. **Rapport qualité-prix** : Le rapport qualité-prix est souvent souligné comme étant très bon, avec des prix raisonnables pour la qualité des plats.
